# Task 2: Visualize hidden representations of a model

In [ ]:
from os import getcwd, pardir, environ as os_environ
from os.path import abspath, exists as path_exists
from os.path import join as path_join
import sys
from datetime import datetime
from os import getcwd, pardir
from os.path import abspath, isdir
from os.path import join as path_join
from time import time

In [ ]:
# Uncomment this if you're viewing this on Google Colab
# download the requirements.txt hosted on a Github public gist and install the required libraries
if 'COLAB_GPU' in os_environ:
    !wget https://gist.githubusercontent.com/CamiloMartinezM/7eb6f86e2642e24776c90433c29133de/raw/368bcd1f53aecfb4b9ca9bf1090730d47872158a/requirements.txt -O requirements.txt
    !pip install -r requirements.txt

    # For t-SNE to run on GPU, using tsnecuda installable on Colab with these commands
    # See: https://inside-machinelearning.com/en/use-tsne-cuda-on-google-colab/
    !pip install tsnecuda
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !wget https://anaconda.org/CannyLab/tsnecuda/3.0.0/download/linux-64/tsnecuda-3.0.0-h1234567_py3.8_73_cuda10.2.tar.bz2
    !tar xvjf tsnecuda-3.0.0-h1234567_py3.8_73_cuda10.2.tar.bz2

    if not path_exists("/usr/local/lib/python3.10/dist-packages/"):
        raise Exception("Please change the cp command param to be the actual directory to Colab's Python dist-packages")
    else:
        !cp -r site-packages/* /usr/local/lib/python3.10/dist-packages/
        !conda install --offline tsnecuda-3.0.0-h1234567_py3.8_73_cuda10.2.tar.bz2

        !echo $LD_LIBRARY_PATH 
        ld_library_paths = os_environ['LD_LIBRARY_PATH'].split(':')
        libfaiss_path = '/content/lib/libfaiss.so'

        for path in ld_library_paths:
            symlink_path = path_join(path, 'libfaiss.so')
            # Check if the path exists before trying to create a symlink
            if isdir(path) and not path_exists(symlink_path):
                !ln -s {libfaiss_path} {symlink_path}
                print(f"Created symlink in {path}")
            else:
                print(f"Path {path} does not exist or symlink already exists.")

In [1]:
parent_dir = abspath(path_join(getcwd(), pardir))

# Add to path the parent directory titled NNTIProject/
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# 'scripts' directory is one level down from the parent directory
# Add it to sys.path if it's not already there
scripts_dir = path_join(parent_dir, "scripts")
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)

# Import task2.py
from task2 import Task2Runner, LANGUAGES, Task2Plotter, RUNNER_CACHE_DIR

# Use Google Colab's content directory if running on Google Colab, otherwise use local cache
CACHE_DIR = "/content/cache/" if 'COLAB_GPU' in os_environ else RUNNER_CACHE_DIR

In [2]:
start_time = time()
print(f"Start time: {datetime.now()}\n")
for dim_reduction in ["t-SNE"]: # ["PCA", "t-SNE"]:
    for model_name in ["facebook/xglm-564M"]:
        # Create a Task2Runner instance for the current model and run it
        runner = Task2Runner(
            LANGUAGES,
            ["devtest"],  # Only analyzing the devtest split
            model_name,
            seq_by_seq=True,
            subset=200,
            cache_dir=CACHE_DIR,
            perform_early_setup=False,
        )
        runner.run()

        # Create a Task2Plotter instance for the current model, run the current dimensionality reduction technique
        # and save each plot to disk for each layer
        for layer in range(0, runner.num_layers + 1):
            print(f"\nRunning {dim_reduction} for layer {layer} of {model_name}...\n")
            plotter = Task2Plotter(
                runner, layer=layer, 
                cache_dir=CACHE_DIR,
                plots_folder="plots_task2_test", 
                use_tsnecuda=True
            )
            plotter.run(dim_reduction=dim_reduction, check_plot_exists=True)

            for ext in ["png", "svg"]:
                plotter.plot(
                    token_repr=True,
                    sentence_repr=True,
                    cmap="Accent",
                    save_to_disk=True,
                    ext=ext,
                    subfolder_for_ext=True,
                    edgecolor="black",
                    linewidth=0.1,
                )

            del plotter
        del runner

print(f"\nTotal elapsed time: {time() - start_time} s")
print(f"End time: {datetime.now()}\n")

Start time: 2024-03-16 16:49:50.261801

Loading dataset for eng_Latn (devtest)... Skipped
Loading dataset for spa_Latn (devtest)... Skipped
Loading dataset for deu_Latn (devtest)... Skipped
Loading dataset for arb_Arab (devtest)... Skipped
Loading dataset for tam_Taml (devtest)... Skipped
Loading dataset for quy_Latn (devtest)... Skipped

Running t-SNE for layer 0 of facebook/xglm-564M...

Initial setup of Token Representations:
	Shape of array for ('eng_Latn', 'devtest'): (6009, 1024)
	Shape of array for ('spa_Latn', 'devtest'): (7424, 1024)
	Shape of array for ('deu_Latn', 'devtest'): (7412, 1024)
	Shape of array for ('arb_Arab', 'devtest'): (7707, 1024)
	Shape of array for ('tam_Taml', 'devtest'): (9271, 1024)
	Shape of array for ('quy_Latn', 'devtest'): (8591, 1024)
Initial setup of Sentence Representations:
	Shape of array for ('eng_Latn', 'devtest'): (200, 1024)
	Shape of array for ('spa_Latn', 'devtest'): (200, 1024)
	Shape of array for ('deu_Latn', 'devtest'): (200, 1024)
	Shap

: 